In [3]:
#Imports
import os
import tensorflow as tf

import sys
sys.path.insert(0, '..')
from utils.federatedAggregation import FederatedAggregation
from utils.reinforcementLearningHelper import *
from utils.federatedLearningHelper import *

In [4]:
#Setup Environments of selected buildings for training, evaluation, and testing

environments, observation_spec, action_spec  = setup_energymanagement_environments(num_buildings=30)

#Check environment setup
print(
    "Batch size:", environments["train"][f"building_1"].batch_size, 
    "/ State Space: {} / Action Space: {}".format(observation_spec.shape[0], action_spec.shape[0]),
    "/ Upper bound: {}".format(round(environments["train"][f"building_1"].action_spec().maximum.item(), 3)),
)

Batch size: 1 / State Space: 22 / Action Space: 1 / Upper bound: 2.3


In [5]:
# Cluster similar buildings (K-Means with DTW)

#Set parameter
num_clusters = 8 # 2, 6, 10, 12, 14, 16, 18

clustered_buildings = load_clustered_buildings(num_clusters)
clustered_buildings

{0: array([ 7, 25], dtype=int64),
 1: array([16, 17, 21], dtype=int64),
 2: array([ 5, 12, 13, 19, 20, 23, 24, 26, 27, 28, 29], dtype=int64),
 3: array([18], dtype=int64),
 4: array([ 3,  4,  9, 14, 15, 22, 30], dtype=int64),
 5: array([1], dtype=int64),
 6: array([6, 8], dtype=int64),
 7: array([ 2, 10, 11], dtype=int64)}

In [6]:
# Setup Agent networks
federated_rounds = 10
batch_size = 128
replay_buffer_capacity = 20000 #-> only <18.000 samples per dataset
initial_collect_steps = 2000 #2000
collect_steps_per_iteration = 30 
num_iterations = 10000 #10000
eval_interval = num_iterations-10 #9999

In [5]:
# FEDERATED LEARNING - Initalization Round 0

tf.compat.v1.reset_default_graph()
global_step = tf.compat.v1.train.get_or_create_global_step()

#Initalize a global model for each Cluster of similar buildings
for cluster in range(num_clusters):
        
        # 1. Build global agent per cluster
        first_building_in_cluster = clustered_buildings[cluster][0]
        global_ppo_agent, global_eval_policy, global_collect_policy = initialize_ppo_agent(
                observation_spec=observation_spec, action_spec=action_spec, global_step=global_step, environments=environments,
                )

        # 2. Initially store weights
        model_dir = os.path.join(os.getcwd(), f"models/ppo/cluster_{cluster}/FLround{0}_c{num_clusters}_AvgAgg")
        os.makedirs(model_dir, exist_ok=True)
        
        save_ppo_weights(global_ppo_agent, model_dir)

In [6]:
#FEDERATED LEARNING - Model training for multiple Rounds

#For each federated round and cluster
for federated_round  in range(federated_rounds):
    for cluster_number, buildings_in_cluster in clustered_buildings.items():

        #Iterate through the buildings per cluster
        print(f"Cluster {cluster_number}: Buildings {buildings_in_cluster} Federated round ---", federated_round+1, f"/ {federated_rounds}")
        local_storage = {
            "actor_weights": [], "value_weights": [], "performance_metrics": []
            }
        
        for building_index in buildings_in_cluster:
            
            #0. Reset global step
            tf.compat.v1.reset_default_graph()
            global_step = tf.compat.v1.train.get_or_create_global_step()
            
            #1. Initalize local agent
            local_ppo_agent, local_eval_policy, local_collect_policy = initialize_ppo_agent(
                observation_spec = observation_spec, action_spec = action_spec,
                global_step = global_step, environments = environments,
                )
            
            #2. Set global weights of this training round to agent (loads the weights of last training)
            model_dir = os.path.join(os.getcwd(), f"models/ppo/cluster_{cluster_number}/FLround{federated_round}_c{num_clusters}_AvgAgg")
            local_ppo_agent = set_weights_to_ppo_agent(local_ppo_agent, model_dir)
            
            #3. Prepare training pipeline: Setup iterator, replay buffer, driver
            local_iterator, local_collect_driver, local_time_step, local_policy_state = setup_rl_training_pipeline(
                local_ppo_agent, environments["train"][f"building_{building_index}"], replay_buffer_capacity,
                local_collect_policy, initial_collect_steps, collect_steps_per_iteration, batch_size
                )
            
            #4. Train, evaluate agent and store weights
            local_ppo_agent, local_storage = local_agent_training_and_evaluation(
                local_iterator, local_collect_driver, local_time_step, local_policy_state, global_step, 
                local_ppo_agent, local_eval_policy, local_storage, building_index, num_iterations, environments, agent_type="ppo"
                )           

        # Performe Federated aggregation
        average_actor_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["actor_weights"], local_storage["performance_metrics"])
        average_value_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["value_weights"], local_storage["performance_metrics"]) 
        
        #Save federated weights for next round (Round + 1)
        model_dir = os.path.join(os.getcwd(), f"models/ppo/cluster_{cluster_number}/FLround{federated_round+1}_c{num_clusters}_AvgAgg")
        os.makedirs(model_dir, exist_ok=True)
        save_federated_ppo_weights(model_dir, average_actor_weights, average_value_weights)

Cluster 0: Buildings [ 7 25] Federated round --- 1 / 10
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Return:  -135205.84
Return:  -133589.28
Cluster 1: Buildings [16 17 21] Federated round --- 1 / 10
Return:  -124940.58
Return:  -123915.92
Return:  -126394.33
Cluster 2: Buildings [ 5 12 13 19 20 23 24 26 27 28 29] Federated round --- 1 / 10
Return:  -112457.555
Return:  -116514.65
Return:  -118429.79
Return:  -109521.1
Return:  -118253.18
Return:  -117463.484
Return:  -118163.02
Return:  -121927.09
Return:  -121653.03
Return:  -113245.74
Return:  -114006.016
Cluster 3: Buildings [18] Federated round --- 1 / 10
Return:  -94414.734
Cluster 4: Buildings [ 3  4  9 14 15 22 30] Federated round --- 1 / 10
Return:  -3234.6978
Return:  -4976.846
Return:  -5114.2344
Return:  -3155.1145
Return:  -3371.4355
Return:  -4507.986
Return:  -3658.0598
Cluster 5: Buildings [1] Federated round --- 1 / 10
Return:  -95373.875
Cluster 6: Buildings [6 8] Federated

In [13]:
# LOCAL REFITTING AND EVALUATION

best_federated_round = 3
num_rounds=3
num_test_iterations = 2000

result_df = pd.DataFrame(columns=['Building', 'Total Profit'])

for cluster_number, buildings_in_cluster in clustered_buildings.items():
    for building_index in buildings_in_cluster:
        
        for round in range(num_rounds):
            print("Cluster: ", cluster_number, " - Building: ", building_index, " - round: ", round)
            
            #0. Reset global step
            tf.compat.v1.reset_default_graph()
            global_step = tf.compat.v1.train.get_or_create_global_step()

            #1. Initalize local agent
            tf_ppo_agent, eval_policy, collect_policy = initialize_ppo_agent(
                observation_spec = observation_spec, action_spec = action_spec,
                global_step = global_step, environments = environments,
                )
            
            #2. Set global weights of this training round to agent (loads the weights of last training)
            model_dir = os.path.join(os.getcwd(), f"models/ppo/cluster_{cluster_number}/FLround{best_federated_round}_c{num_clusters}_AvgAgg")
            tf_agent = set_weights_to_ppo_agent(tf_ppo_agent, model_dir)

            #3. Prepare training pipeline: Setup iterator, replay buffer, driver
            iterator, collect_driver, time_step, policy_state = setup_rl_training_pipeline(
                tf_ppo_agent, environments["train"][f"building_{building_index}"], replay_buffer_capacity,
                collect_policy, initial_collect_steps, collect_steps_per_iteration, batch_size
                )
            
            #4. Setup wandb logging
            artifact = initialize_wandb_logging(name=f"Exp_building{building_index}_rd{round}", num_iterations=num_iterations)
            
            #5. Train, evaluate agent and store weights
            result_df, metrics = agent_training_and_evaluation(global_step, num_test_iterations, collect_driver, 
                time_step, policy_state, iterator, tf_ppo_agent, eval_policy, building_index, result_df, eval_interval, environments)
            
            #6. End and log wandb
            end_and_log_wandb(metrics, artifact)

Cluster:  0  - Building:  7  - round:  0


c:\Users\rs1044\Documents\GitHub\Federated-Reinforcement-Learning-for-Battery-Charging\src\notebooks\..\utils\reinforcementLearningHelper.py:353: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame({'Building': [building_index], 'Total Profit': [wandb.summary["Final Profit"]]})], ignore_index=True)


Building:  7  - Total Profit:  -800.1376223306403
Cluster:  0  - Building:  7  - round:  1
Building:  7  - Total Profit:  -796.8496815705988
Cluster:  0  - Building:  7  - round:  2
Building:  7  - Total Profit:  -796.8481795921967
Cluster:  0  - Building:  25  - round:  0
Building:  25  - Total Profit:  -1441.4853373078788
Cluster:  0  - Building:  25  - round:  1
Building:  25  - Total Profit:  -1441.4853373078788
Cluster:  0  - Building:  25  - round:  2
Building:  25  - Total Profit:  -1441.4852886991061
Cluster:  1  - Building:  16  - round:  0
Building:  16  - Total Profit:  -1254.4983150490305
Cluster:  1  - Building:  16  - round:  1
Building:  16  - Total Profit:  -1254.524615937471
Cluster:  1  - Building:  16  - round:  2
Building:  16  - Total Profit:  -1254.522947096724
Cluster:  1  - Building:  17  - round:  0
Building:  17  - Total Profit:  -1847.5232586058492
Cluster:  1  - Building:  17  - round:  1
Building:  17  - Total Profit:  -1847.548282469304
Cluster:  1  - Buil

In [14]:
# Save results
result_df.rename(columns={'Total Profit': 'Profit'}, inplace=True)
result_df['Setup'] = 'PPO_FL'
result_df.index.name = 'Building_nr'
result_df.reset_index(inplace=True, drop=True)
os.makedirs('results', exist_ok=True)
result_df.to_csv(f'results/PPO_FL_results_setting_A_bR3_rF2k.csv', index=False)
print("Final reuslt: ", result_df["Profit"].sum())

Final reuslt:  -107489.8324187874


In [15]:
import requests

def send_telegram_message(bot_token, chat_id, message):
    """Send a message to a Telegram chat via the Bot API."""
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": message,
        "parse_mode": "Markdown"
    }
    response = requests.post(url, json=payload)
    return response.json()

# Use the function
bot_token = os.getenv('TELEGRAM_BOT_TOKEN')
chat_id = os.getenv('TELEGRAM_CHAT_ID')
message = f"Script PPO FL ist fertig am PC .81!"

result = send_telegram_message(bot_token, chat_id, message)
print(result)

{'ok': True, 'result': {'message_id': 44, 'from': {'id': 7071194232, 'is_bot': True, 'first_name': 'Reinforcement Learning', 'username': 'FederatedRL_Bot'}, 'chat': {'id': 5493937056, 'first_name': 'Jonas', 'last_name': 'Sievers', 'username': 'JonasSievers', 'type': 'private'}, 'date': 1715345293, 'text': 'Script PPO FL ist fertig am PC .81!'}}
